In [20]:
import pandas
import numpy
data = pandas.read_csv('gym_MachiKoro/envs/base/card.csv', index_col='name')
data

,card_type,price,description,value_to_activate,activated_from,income,income_from,income_times,card_type_in_effect
name,,,,,,,,,
Train Station,Big Construction,4,You may roll 1 or 2 dice on your turn,NaN,NaN,NaN,NaN,NaN,NaN
Shopping Mall,Big Construction,10,Increase the number of coins you get for your ...,NaN,NaN,NaN,NaN,NaN,NaN
Amusement Park,Big Construction,16,"If you roll doubles, take another turn immedia...",NaN,NaN,NaN,NaN,NaN,NaN
Radio Tower,Big Construction,22,You may re-roll your dice once each turn,NaN,NaN,NaN,NaN,NaN,NaN
Wheat Field,Field,1,Receive 1 coin from the bank in anyone's turn,1,Anyone,1.0,Bank,Once,NaN
Livestock Farm,Farm,1,Receive 1 coin from the bank in anyone's turn,2,Anyone,1.0,Bank,Once,NaN
Bakery,Store,1,Receive 1 coin from the bank if it's your turn,"2,3",You,1.0,Bank,Once,NaN
Cafe,F&B Service,2,Receive 1 coin from any player who rolls this ...,3,Dice Roller,1.0,Dice Roller,Once,NaN
Convenience Store,Store,2,Receive 3 coins from the bank if it's your turn,4,You,3.0,Bank,Once,NaN


In [66]:
data[(data['activated_from'] == 'Anyone') | (data['activated_from'] == 'You')]

,card_type,price,description,value_to_activate,activated_from,income,income_from,income_times,card_type_in_effect
name,,,,,,,,,
Wheat Field,Field,1,Receive 1 coin from the bank in anyone's turn,1,Anyone,1.0,Bank,Once,NaN
Livestock Farm,Farm,1,Receive 1 coin from the bank in anyone's turn,2,Anyone,1.0,Bank,Once,NaN
Bakery,Store,1,Receive 1 coin from the bank if it's your turn,"2,3",You,1.0,Bank,Once,NaN
Convenience Store,Store,2,Receive 3 coins from the bank if it's your turn,4,You,3.0,Bank,Once,NaN
Forest,Industry,3,Receive 1 coin from the bank in anyone's turn,5,Anyone,1.0,Bank,Once,NaN
Stadium,Big Construction,6,Receive 2 coins from each player if it's your ...,6,You,2.0,Each Player,Each Player,NaN
TV Station,Big Construction,7,Receive 5 coins from one player of your choice...,6,You,5.0,Chosen Player,Once,NaN
Business Complex,Big Construction,8,You may exchange one 'non-Big_Construction' ca...,6,You,NaN,Exchange,Once,NaN
Cheese Factory,Factory,5,Receive 3 coins from the bank for every 'Farm'...,7,You,3.0,Bank,Each Card_type,Farm


In [1]:
from copy import deepcopy


class Card:
    def __init__(self, card_type: str, name: str, price: int, description: str):
        self.__card_type = card_type # Cái biểu tượng bên cạnh tên thẻ
        self.__name = name
        self.__price = price # Giá để mua thẻ
        self.__description = description

    @property
    def card_type(self):
        return self.__card_type
    
    @property
    def name(self):
        return self.__name

    @property
    def price(self):
        return self.__price

    @property
    def description(self):
        return self.__description

    def convert_to_dict(self):
        return deepcopy({
            'card_type': self.__card_type,
            'name': self.__name,
            'price': self.__price
        })
        

class Important_Land_Card(Card):
    def __init__(self, card_type: str, name: str, price: int, description: str):
        super().__init__(card_type, name, price, description)

class Support_Card(Card):
    def __init__(self, card_type: str, name: str, price: int, description: str, value_to_activate: list, activate_from: str,
                income: None or int, income_from: str, income_times: str, card_type_in_effect: None or str):
        super().__init__(card_type, name, price, description)
        self.__value_to_activate = value_to_activate.copy()
        self.__activate_from = activate_from
        self.__income = income
        self.__income_from = income_from
        self.__income_times = income_times
        self.__card_type_in_effect = card_type_in_effect
    
    @property
    def value_to_activate(self):
        return self.__value_to_activate.copy()
    
    @property
    def activate_from(self):
        return self.__activate_from
    
    @property
    def income(self):
        return self.__income
    
    @property
    def income_from(self):
        return self.__income_from
    
    @property
    def income_times(self):
        return self.__income_times
    
    @property
    def card_type_in_effect(self):
        return self.__card_type_in_effect

    def convert_to_dict(self):
        return deepcopy({
            'card_type': self.card_type,
            'name': self.name,
            'price': self.price,
            'value_to_activate': self.__value_to_activate.copy(),
            'activate_from': self.__activate_from,
            'income': self.__income,
            'income_from': self.__income_from,
            'income_times': self.__income_times,
            'card_type_in_effect': self.__card_type_in_effect
        })

In [ ]:
list_Important_Land_Card = []
for name in data.index[:4]:
    list_Important_Land_Card.append(Important_Land_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name]))

In [ ]:
list_Support_Card = []
for name in data.index[4:]:
    try:
        income = int(data['income'][name])
    except:
        income = None
    list_Support_Card.append(Support_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name], [int(a) for a in data['value_to_activate'][name].split(',')], data['activate_from'][name], income, data['income_from'][name], data['income_times'][name], data['card_type_in_effect'][name]))

In [ ]:
for card in list_Important_Land_Card:
    print('###', card.card_type, '***card_type')
    print(card.name, '***name')
    print(card.price, '***price')
    print(card.description, '***description')

In [ ]:
for card in list_Support_Card:
    print('###', card.card_type, '***card_type')
    print(card.name, '***name')
    print(card.price, '***price')
    print(card.description, '***description')
    print(card.value_to_activate, '***value_to_activate')
    print(card.activate_from, '***activate_from')
    print(card.income, '***income')
    print(card.income_from, '***income_from')
    print(card.income_times, '***income_times')
    print(card.card_type_in_effect, '***card_type_in_effect')

In [ ]:
temp = [card.name for card in list_Important_Land_Card]
print(temp)

In [ ]:
from copy import deepcopy
import pandas
# from gym_MachiKoro.envs.base.card import Support_Card, Important_Land_Card


class Board:
    def __init__(self):
        temp = ['Wheat Field', 'Livestock Farm', 'Bakery', 'Cafe', 'Convenience Store', 'Forest', 'Stadium', 'TV Station', 'Business Complex',
                'Cheese Factory', 'Furniture Factory', 'Mine', 'Family Restaurant', 'Apple Orchard', 'Vegetable Market']

        self.__support_cards = {}
        for name in temp:
            if name not in ['Stadium', 'TV Station', 'Business Complex']:
                self.__support_cards[name] = 6
            else:
                self.__support_cards[name] = 4
        
        data = pandas.read_csv('gym_MachiKoro/envs/base/card.csv', index_col='name')

        self.__support_cards_object = {}
        for name in temp:
            try:
                income = int(data['income'][name])
            except:
                income = None

            self.__support_cards_object[name] = Support_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name],[int(a) for a in data['value_to_activate'][name].split(',')],
                                                            data['activate_from'][name], income, data['income_from'][name], data['income_times'][name], data['card_type_in_effect'][name])

    @property
    def support_cards(self):
        return deepcopy(self.__support_cards)

    @property
    def support_cards_object(self):
        return deepcopy(self.__support_cards_object)

In [ ]:
board = Board()

In [ ]:
for key in board.support_cards_object.keys():
    print(type(board.support_cards_object[key]), board.support_cards_object[key].convert_to_dict())

In [ ]:
print(board.support_cards)

In [2]:
from copy import deepcopy
import pandas
# from gym_MachiKoro.envs.base.card import Support_Card, Important_Land_Card


class Player:
    def __init__(self, name):
        self.__name = name
        self.__full_action = []
        self.__amount_action_space = 0
        self.reset()

    def reset(self):
        data = pandas.read_csv('gym_MachiKoro/envs/base/card.csv', index_col='name')
        temp = ['Wheat Field', 'Livestock Farm', 'Bakery', 'Cafe', 'Convenience Store', 'Forest', 'Stadium', 'TV Station', 'Business Complex',
                'Cheese Factory', 'Furniture Factory', 'Mine', 'Family Restaurant', 'Apple Orchard', 'Vegetable Market']

        self.__support_cards = {}
        for name in temp:
            self.__support_cards[name] = 0

        self.__support_cards_object = {}
        for name in temp:
            try:
                income = int(data['income'][name])
            except:
                income = None

            self.__support_cards_object[name] = Support_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name],[int(a) for a in data['value_to_activate'][name].split(',')],
                                                            data['activate_from'][name], income, data['income_from'][name], data['income_times'][name], data['card_type_in_effect'][name])

        temp = ['Train Station', 'Shopping Mall', 'Amusement Park', 'Radio Tower']
        self.__important_land_cards = {}
        for name in temp:
            self.__important_land_cards[name] = 0
        
        self.__important_land_cards_object = {}
        for name in temp:
            self.__important_land_cards_object[name] = Important_Land_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name])

    @property
    def name(self):
        return self.__name
    
    @property
    def support_cards(self):
        return deepcopy(self.__support_cards)

    @property
    def support_cards_object(self):
        return deepcopy(self.__support_cards_object)

    @property
    def important_land_cards(self):
        return deepcopy(self.__important_land_cards)
    
    @property
    def important_land_cards_object(self):
        return deepcopy(self.__important_land_cards_object)

    @property
    def amount_action_space(self):
        return self.__amount_action_space

In [ ]:
player = Player('An')

In [ ]:
for key in player.important_land_cards_object.keys():
    print(type(player.important_land_cards_object[key]), player.important_land_cards_object[key].convert_to_dict())

In [ ]:
print(player.support_cards)

In [1]:
import random
def roll_the_dice(number_of_dice=1):
    if number_of_dice == 1:
        return random.randint(1,6)
        
    elif number_of_dice == 2:
        return random.randint(1,6), random.randint(1,6)

In [31]:
from collections import Counter
a = Counter([roll_the_dice(2) for i in range(1000000)])
a.__len__()

36

In [10]:
temp = ['A', 'B']
turn = 0
for i in range(turn-1,turn-temp.__len__(),-1):
    print(temp[i])

B


In [12]:
a = {
    'a': 3,
    'b': 4,
    'c': 5,
    'd': 6
}

b = {item for item in a.items() if item[1] % 2 == 1}
b

{('a', 3), ('c', 5)}